In [4]:
import numpy as np
import tensorflow.contrib.distributions as ds
from sklearn.utils import shuffle
import tensorflow as tf
import argparse
import ram

In [5]:
sess = tf.InteractiveSession()

In [15]:
# side of full glimpse will actually be glimpse_size * 2 + 1
glimpse_size = 4
num_resolutions = 3
batch_size = 9
length = 100
num_tfs = 2

In [241]:
def make_dna_seq(batch_size, length):
    one_hot_bases = np.eye(4)
    sample_indices = np.random.randint(0, 4, [batch_size, length])
    return one_hot_bases[sample_indices]

def make_atac_seq(batch_size, length):
    return np.random.randint(0, 20, [batch_size, length, 1])
    
def make_chip_seq(batch_size, num_tfs):
    return np.random.randint(0, 2, [batch_size, num_tfs])

def get_glimpses(data, num_resolutions):
    glimpses = []
    for i in range(num_resolutions):
        resolution = 2**i
        glimpse = tf.nn.pool(
            input=data,
            window_shape=[resolution],
            strides=[resolution],
            pooling_type='MAX',
            padding='SAME')
        glimpses.append(glimpse)
    return glimpses

def index_glimpses(dna, location, num_resolutions, glimpses, glimpse_size, length, batch_size):
    to_concatenate = []
    for i in range(num_resolutions):
        glimpse = glimpses[i]
        # add glimpse_size to location_index
        # because glimpses will be padded with glimpse_size values on each side
        location_index = tf.to_int32(location / 2.0**i) + glimpse_size
        start_index = location_index - glimpse_size
        boolean_mask = get_boolean_mask(glimpse_size, start_index, glimpse.shape[1])
        padded_glimpse = get_padded_glimspe(glimpse, glimpse_size)
        if i == 0:
            padded_dna = get_padded_dna(dna, glimpse_size)
            dna_boolean_mask = tf.squeeze(tf.stack([boolean_mask]*4, axis=-1))
            sliced_dna = tf.boolean_mask(padded_dna, dna_boolean_mask)
            print(boolean_mask)
            print(dna_boolean_mask)
            print(sliced_dna)
            sliced_dna = tf.reshape(sliced_dna, [batch_size, glimpse_size * 2, 4])
            to_concatenate.append(tf.to_int64(sliced_dna))
        sliced_glimpse = tf.boolean_mask(padded_glimpse, boolean_mask)
        sliced_glimpse = tf.reshape(sliced_glimpse, [batch_size, glimpse_size * 2 + 1, 1])
        to_concatenate.append(sliced_glimpse)
#     print(to_concatenate)
    return tf.concat(to_concatenate, axis=-1)
        
def get_boolean_mask(glimpse_size, start_index, length):
    curr_index = start_index
    padded_size = length + 2 * glimpse_size
    index_mask = tf.one_hot(indices=curr_index, depth=padded_size, axis=1)
    for i in range(glimpse_size * 2 - 1):
        curr_index += 1
        index_mask += tf.one_hot(indices=curr_index, depth=padded_size, axis=1)
    return index_mask > 0
    
def get_padded_glimspe(glimpse, glimpse_size):
    return tf.pad(glimpse, paddings=[[0, 0], [glimpse_size, glimpse_size], [0, 0]], constant_values=-1)

def get_padded_dna(dna, glimpse_size):
    return get_padded_glimspe(dna, glimpse_size)


In [242]:
dna = make_dna_seq(batch_size, length)
atac = make_atac_seq(batch_size, length)
chip = make_chip_seq(batch_size, num_tfs)
input_ = np.concatenate([dna, atac], axis=-1)
input_.shape

(9, 100, 5)

In [243]:
location = np.array([[20], [40], [99]] * 3)

glimpses = get_glimpses(atac, num_resolutions)
# index_glimpses(dna, num_resolutions, glimpses, 50, glimpse_size).eval()
g = index_glimpses(dna, location, num_resolutions, glimpses, glimpse_size, length, batch_size)
g.eval(), g.shape

0
1
2
3
4
5
6
Tensor("Greater_167:0", shape=(9, 108, 1), dtype=bool)
Tensor("Squeeze_31:0", shape=(9, 108, 4), dtype=bool)
Tensor("boolean_mask_210/Gather:0", shape=(?,), dtype=float64)
0
1
2
3
4
5
6
0
1
2
3
4
5
6


InvalidArgumentError: Input to reshape is a tensor with 72 values, but the requested shape has 81
	 [[Node: Reshape_153 = Reshape[T=DT_INT64, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](boolean_mask_211/Gather, Reshape_153/shape)]]

Caused by op 'Reshape_153', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/Gunjan/.virtualenvs/deeprl/lib/python3.4/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/Gunjan/.virtualenvs/deeprl/lib/python3.4/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/Gunjan/.virtualenvs/deeprl/lib/python3.4/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.inst

In [223]:
print(glimpses[0])

Tensor("max_pool_216/Squeeze:0", shape=(9, 100, 1), dtype=int64)
